In [1]:
# importing the libraries
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Dense,Flatten,Lambda,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob

In [2]:
#re-size all the images to this 
Image_Size = [224,224]

train_path = "/content/drive/MyDrive/Skin_Cancer_train_data"
valid_path = "/content/drive/MyDrive/Skin_Cancer_test_data"

In [3]:
# import the Vgg16 library as shown below and add preprocessing layerto the front of VGG
# here we will be using imagenet weights. if it is not given weights will not be using weights for image classification using vgg16 in imagenet competiton 
# include_top = False means we are removing vgg16 layers from flatten till output layer so that we add our own layers 
# by giving input shape we have added our own input layer

vgg16 = VGG16(input_shape=Image_Size + [3],weights='imagenet',include_top=False) 


58892288/58889256 [==============================] - 1s 0us/step


In [4]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
#don't train existing weights 
for layer in vgg16.layers:
  layer.trainable = False
  

In [6]:
#useful for getting number of output classes
folders = glob("/content/drive/MyDrive/Skin_Cancer_train_data/*")

In [7]:
folders

['/content/drive/MyDrive/Skin_Cancer_train_data/malignant',
 '/content/drive/MyDrive/Skin_Cancer_train_data/benign']

In [8]:
# our own layer 
x = Flatten()(vgg16.output)

In [35]:
prediction = Dense(len(folders),activation='softmax')(x)

# create a model object 
model = Model(inputs = vgg16.input,outputs = prediction)

In [36]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [37]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2,zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
training_set = train_datagen.flow_from_directory("/content/drive/MyDrive/Skin_Cancer_train_data",
                                                 target_size =(224,224),
                                                 batch_size =32,
                                                 class_mode='categorical')

Found 2637 images belonging to 2 classes.


In [40]:
testing_set = test_datagen.flow_from_directory("/content/drive/MyDrive/Skin_Cancer_test_data",
                                                 target_size =(224,224),
                                                 batch_size =32,
                                                 class_mode='categorical')

Found 660 images belonging to 2 classes.


In [41]:
model.fit_generator(training_set,validation_data=testing_set,epochs=2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
83/83 [==============================] - 1750s 21s/step - loss: 0.7486 - accuracy: 0.6759 - val_loss: 0.3680 - val_accuracy: 0.8348
Epoch 2/2
83/83 [==============================] - 48s 576ms/step - loss: 0.3537 - accuracy: 0.8388 - val_loss: 0.3363 - val_accuracy: 0.8530


In [42]:
import numpy as np 
from tensorflow.keras.preprocessing import image
test_image = image.load_img("/content/drive/MyDrive/Skin_Cancer_test_data/benign/1003.jpg",target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)


In [43]:
result

array([[0.918031, 0.081969]], dtype=float32)

In [44]:
a = np.argmax(model.predict(test_image),axis=1)
a

array([0])

In [45]:
if a==0:
  print("The image classified is Benign")
else:
  print("The image classified is Malignant")  

The image classified is Benign
